In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    try:
        nonce = generate_nonce()
        timestamp = get_timestamp()
        sign_str = build_sign_str(APPID, nonce, timestamp)
        signature = calculate_signature(SECRET, sign_str)

        headers = {
            'nonce': nonce,
            'timestamp': str(timestamp),
            'appid': APPID,
            'signature': signature,
            'Accept': "*/*",
            'Accept-Encoding': "gzip, deflate, br",
            'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
            'Connection': "keep-alive",
        }

        url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json().get('Data')
            if not data:
                return None, f"无数据返回"
            return data, None
        else:
            return None, f"HTTP状态码: {response.status_code}, 响应内容: {response.text}"
    except Exception as e:
        return None, f"请求出错: {str(e)}"

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    try:
        nonce = generate_nonce()
        timestamp = get_timestamp()
        sign_str = build_sign_str(APPID, nonce, timestamp)
        signature = calculate_signature(SECRET, sign_str)

        headers = {
            'nonce': nonce,
            'timestamp': str(timestamp),
            'appid': APPID,
            'signature': signature,
            'Accept': "*/*",
            'Accept-Encoding': "gzip, deflate, br",
            'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
            'Connection': "keep-alive",
        }

        url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            name = response.json().get('Data', {}).get('EdbName')
            if not name:
                return None, "无法获取指标名称"
            return name, None
        else:
            return None, f"HTTP状态码: {response.status_code}, 响应内容: {response.text}"
    except Exception as e:
        return None, f"请求出错: {str(e)}"

def main():
    indicator_ids = ['C2505291042239748','C2505291113421348','M0067855','C2302232479','wsurcndp index','C2505290911199345',
                     'ID00187915','C2506101521091792_250610152543','C2506101547229521','C2506181507222042',
                     'C2307286686','C2506181327288056','C2505210952028892','C2506201034377510']

    # 存储失败的指标信息
    failed_indicators = {}
    data_frames = {}

    for indicator_id in indicator_ids:
        # 获取指标名称
        name, name_error = fetch_indicator_name(indicator_id)
        if name_error:
            failed_indicators[indicator_id] = f"获取指标名称失败: {name_error}"
            continue

        # 获取指标数据
        data, data_error = fetch_indicator_details(indicator_id)
        if data_error:
            failed_indicators[indicator_id] = f"获取数据失败: {data_error}"
            continue

        # 处理数据
        try:
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            df = df[['Value']].rename(columns={'Value': name})
            data_frames[indicator_id] = df
        except Exception as e:
            failed_indicators[indicator_id] = f"数据处理失败: {str(e)}"

    # 输出失败的指标信息
    if failed_indicators:
        print("\n获取失败的指标信息:")
        for indicator_id, error in failed_indicators.items():
            print(f"指标ID: {indicator_id}")
            print(f"错误原因: {error}\n")

    # 保存成功获取的数据
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print("\n成功获取的数据信息:")
        print(result_df.info())
        result_df.to_excel("data_input/SC期货指数-Brent原油期货价格.xlsx")
        print("\n数据已保存至 'SC期货指数-Brent原油期货价格.xlsx'")
    else:
        print("\n警告: 没有成功获取任何数据!")



if __name__ == "__main__":
    main()


成功获取的数据信息:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14934 entries, 1985-01-21 to 2025-12-26
Data columns (total 14 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   SC期货指数-Brent原油期货价格                                1842 non-null   float64
 1   中国10年期国债收益率-美国10年期国债实际收益率20日环差                    4322 non-null   float64
 2   人民币汇率收盘价                                          8318 non-null   float64
 3   汽油出口利润（华东-新加坡）                                    2034 non-null   float64
 4   WESTPAC中国经济惊喜指数                                   1332 non-null   float64
 5   SC仓单（百万桶）20日环差                                    1814 non-null   float64
 6   中国主营炼油企业利润                                        380 non-null    float64
 7   SC期货指数-Brent原油期货价格Seasonal/F0.2/5DMA(预测/过去2年季节性)  1885 non-null   float64
 8   Kpler-中国原油浮仓超季节性/3年                               355 non-null    f


数据已保存至 'SC期货指数-Brent原油期货价格.xlsx'
